# Linking a bqplot to a datagrid widget

## Load data

In [1]:
import json

import numpy as np
import pandas as pd

n = 10000

df = pd.DataFrame({
    'x': np.random.randn(n),
    'y': np.random.randn(n),
    'Dates': pd.date_range(end=pd.Timestamp('today'), periods=n)
})
jsontable = json.loads(df.to_json(orient='table'))

## Create scales

In [2]:
from bqplot import DateScale, DateColorScale, ColorScale, LinearScale

min_date = df['Dates'][0]
max_date = df['Dates'][n-1]

scale_x = LinearScale()
scale_y = LinearScale()
xy_color_scale = ColorScale(min=-5, max=5)
date_scale = DateScale(min=min_date, max=max_date)
date_color_scale = DateColorScale(min=min_date, max=max_date, scheme='viridis')

## Create bplot with a brush selector

In [3]:
from bqplot import Axis, ColorAxis, Figure, ScatterGL
from bqplot.interacts import BrushIntervalSelector, PanZoom

scatter = ScatterGL(
    x=df['x'], y=df['y'], color=df['Dates'],
    scales={
        'x': scale_x, 'y': scale_y, 
        'color': date_color_scale
    },
    marker='circle',
    stroke='black'
)

ax_x = Axis(scale=scale_x, grid_lines='solid', tick_format='0.2f', label='X')
ax_y = Axis(scale=scale_y, orientation='vertical', tick_format='0.2f', grid_lines='solid', label='Y')

fig = Figure(marks=[scatter], axes=[ax_x, ax_y], title='My Plot', legend_location='bottom-right')

selector = BrushIntervalSelector(marks=[scatter], scale=scale_x)

## Create datagrid

In [4]:
from py2vega.functions.type_coercing import toDate
from py2vega.functions.date_time import datetime

from ipydatagrid import Expr, DataGrid, TextRenderer, BarRenderer

text_renderer = TextRenderer(
    background_color=Expr('default_value')
)

renderers = {
    'x': text_renderer,
    'Dates': BarRenderer(
        bar_value=date_scale,
        bar_color=date_color_scale,
        format='%Y/%m/%d',
        format_type='time'
    ),
}

grid = DataGrid(data=jsontable, base_row_size=30, base_column_size=300, renderers=renderers)
grid.transform([{'type': 'sort', 'columnIndex': 2, 'desc': True}])

In [5]:
from ipywidgets import HBox

grid.layout.width = '50%'

HBox((fig, grid))

## Add interactions

### Add mouse selector to the figure

In [6]:
scatter.selected = []
fig.interaction = selector

### Filter grid on selection

In [7]:
def update_renderer(*args, **kwargs):
    grid.revert()
    grid.transform([{'type': 'filter', 'operator': 'between', 'columnIndex': 1, 'value': list(selector.selected)}])

selector.observe(update_renderer, 'selected')

In [ ]:
selector.unobserve(update_renderer, 'selected')
grid.revert()

### Change grid highlighting on selection

In [ ]:
def update_renderer(*args, **kwargs):
    text_renderer.background_color.value = "'green' if value > {} and value < {} else default_value".format(*selector.selected)

selector.observe(update_renderer, 'selected')